In [1]:
import pandas as pd
import numpy as np
import pyodbc
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns

# Data Loading and Exploration

In [2]:
# Load the datasets
education_df = pd.read_csv('E:\\DEPI\\final_project\\HR\\EducationLevel.csv')
employee_df = pd.read_csv('E:\\DEPI\\final_project\\HR\\Employee.csv')
performance_df = pd.read_csv('E:\\DEPI\\final_project\\HR\\PerformanceRating.csv')
rating_df = pd.read_csv('E:\\DEPI\\final_project\\HR\\RatingLevel.csv')
satisfaction_df = pd.read_csv('E:\\DEPI\\final_project\\HR\\SatisfiedLevel.csv')

# Displaying the first few rows of each dataset to inspect the structure
education_df_head = education_df.head()
employee_df_head = employee_df.head()
performance_df_head = performance_df.head()
rating_df_head = rating_df.head()
satisfaction_df_head = satisfaction_df.head()

education_df_head, employee_df_head, performance_df_head, rating_df_head, satisfaction_df_head

(   EducationLevelID            EducationLevel
 0                 1  No Formal Qualifications
 1                 2              High School 
 2                 3                Bachelors 
 3                 4                  Masters 
 4                 5                 Doctorate,
   EmployeeID   FirstName LastName      Gender  Age BusinessTravel  \
 0  3012-1A41    Leonelle    Simco      Female   30    Some Travel   
 1  CBCB-9C9D     Leonerd    Aland        Male   38    Some Travel   
 2  95D7-1CE9       Ahmed    Sykes        Male   43    Some Travel   
 3  47A0-559B  Ermentrude   Berrie  Non-Binary   39    Some Travel   
 4  42CC-040A       Stace   Savege      Female   29    Some Travel   
 
         Department  DistanceFromHome (KM) State                Ethnicity  ...  \
 0            Sales                     27    IL                    White  ...   
 1            Sales                     23    CA                    White  ...   
 2  Human Resources                     29    CA 

In [3]:
# Check for missing values in all datasets
missing_education = education_df.isnull().sum()
missing_employee = employee_df.isnull().sum()
missing_performance = performance_df.isnull().sum()
missing_rating = rating_df.isnull().sum()
missing_satisfaction = satisfaction_df.isnull().sum()

missing_education, missing_employee, missing_performance, missing_rating, missing_satisfaction


(EducationLevelID    0
 EducationLevel      0
 dtype: int64,
 EmployeeID                 0
 FirstName                  0
 LastName                   0
 Gender                     0
 Age                        0
 BusinessTravel             0
 Department                 0
 DistanceFromHome (KM)      0
 State                      0
 Ethnicity                  0
 Education                  0
 EducationField             0
 JobRole                    0
 MaritalStatus              0
 Salary                     0
 StockOptionLevel           0
 OverTime                   0
 HireDate                   0
 Attrition                  0
 YearsAtCompany             0
 YearsInMostRecentRole      0
 YearsSinceLastPromotion    0
 YearsWithCurrManager       0
 dtype: int64,
 PerformanceID                      0
 EmployeeID                         0
 ReviewDate                         0
 EnvironmentSatisfaction            0
 JobSatisfaction                    0
 RelationshipSatisfaction           0
 Train

In [4]:
# Calculate the number of unique values for each column in the employee and performance_rating dataframes
unique_values_employee = employee_df.nunique()
unique_values_performance = performance_df.nunique()

# Display the results
print("Unique Values in Employee Dataset:")
print(unique_values_employee)

print("\nUnique Values in Performance Rating Dataset:")
print(unique_values_performance)


Unique Values in Employee Dataset:
EmployeeID                 1470
FirstName                  1334
LastName                   1441
Gender                        4
Age                          34
BusinessTravel                3
Department                    3
DistanceFromHome (KM)        45
State                         3
Ethnicity                     7
Education                     5
EducationField                9
JobRole                      13
MaritalStatus                 3
Salary                     1455
StockOptionLevel              4
OverTime                      2
HireDate                   1192
Attrition                     2
YearsAtCompany               11
YearsInMostRecentRole        11
YearsSinceLastPromotion      11
YearsWithCurrManager         11
dtype: int64

Unique Values in Performance Rating Dataset:
PerformanceID                      6709
EmployeeID                         1280
ReviewDate                         2771
EnvironmentSatisfaction               5
JobSatisfa

In [5]:
# Return the unique values for specified columns in the employee dataframe
unique_values_columns = {
    'Gender': employee_df['Gender'].unique(),
    'BusinessTravel': employee_df['BusinessTravel'].unique(),
    'Department': employee_df['Department'].unique(),
    'JobRole': employee_df['JobRole'].unique(),
    'State': employee_df['State'].unique(),
    'Ethnicity': employee_df['Ethnicity'].unique(),
    'EducationField': employee_df['EducationField'].unique()
}

# Display the unique values for the specified columns
for column, values in unique_values_columns.items():
    print(f"Unique values in {column}:")
    print(values)
    print("\n")


Unique values in Gender:
['Female' 'Male' 'Non-Binary' 'Prefer Not To Say']


Unique values in BusinessTravel:
['Some Travel' 'No Travel ' 'Frequent Traveller']


Unique values in Department:
['Sales' 'Human Resources' 'Technology']


Unique values in JobRole:
['Sales Executive' 'HR Business Partner' 'Engineering Manager' 'Recruiter'
 'Data Scientist' 'Machine Learning Engineer' 'Manager'
 'Software Engineer' 'Senior Software Engineer' 'Sales Representative'
 'Analytics Manager' 'HR Executive' 'HR Manager']


Unique values in State:
['IL' 'CA' 'NY']


Unique values in Ethnicity:
['White' 'Asian or Asian American' 'Mixed or multiple ethnic groups'
 'Black or African American' 'Native Hawaiian ' 'Other '
 'American Indian or Alaska Native']


Unique values in EducationField:
['Marketing' 'Marketing ' 'Computer Science' 'Technical Degree'
 'Information Systems' 'Other' 'Economics' 'Human Resources'
 'Business Studies']




# Preprocess Education Field 

In [6]:
# Remove spaces from the 'EducationField' column
employee_df['EducationField'] = employee_df['EducationField'].str.strip()

# Check the unique values again to ensure the spaces have been removed
unique_education_field = employee_df['EducationField'].unique()

print(unique_education_field)


['Marketing' 'Computer Science' 'Technical Degree' 'Information Systems'
 'Other' 'Economics' 'Human Resources' 'Business Studies']


# Calculate Attrition year then drop from performance rating reviews out of date range

In [7]:
# Add a new column 'AttritionYear' by adding the year of the 'HireDate' to 'YearsAtCompany'
employee_df['HireYear'] = pd.to_datetime(employee_df['HireDate']).dt.year
employee_df['AttritionYear'] = employee_df['HireYear'] + employee_df['YearsAtCompany']

In [8]:
# Convert 'ReviewDate' in performance rating dataframe to datetime for comparison
performance_df['ReviewDate'] = pd.to_datetime(performance_df['ReviewDate'])

# Merge the employee_df with performance_df to get the hire and attrition year for each employee
performance_merged = pd.merge(performance_df, employee_df[['EmployeeID', 'HireYear', 'AttritionYear']], on='EmployeeID', how='left')

# Filter out rows where the review date is out of the employee's hire and attrition year range
performance_df_filtered = performance_merged[
    (performance_merged['ReviewDate'].dt.year >= performance_merged['HireYear']) &
    (performance_merged['ReviewDate'].dt.year <= performance_merged['AttritionYear'])
]

# Drop the extra columns ('HireYear' and 'AttritionYear') from the filtered dataframe
performance_df_filtered = performance_df_filtered.drop(columns=['HireYear', 'AttritionYear'])

# Assign the filtered dataframe back to performance_df
performance_df = performance_df_filtered


In [9]:
print("\nUnique Values in Performance Rating Dataset before filtering:")
print(unique_values_performance)
unique_values_performance = performance_df.nunique()
print("\nUnique Values in Performance Rating Dataset after filtering:")
print(unique_values_performance)


Unique Values in Performance Rating Dataset before filtering:
PerformanceID                      6709
EmployeeID                         1280
ReviewDate                         2771
EnvironmentSatisfaction               5
JobSatisfaction                       5
RelationshipSatisfaction              5
TrainingOpportunitiesWithinYear       3
TrainingOpportunitiesTaken            4
WorkLifeBalance                       5
SelfRating                            3
ManagerRating                         4
dtype: int64

Unique Values in Performance Rating Dataset after filtering:
PerformanceID                      5136
EmployeeID                         1235
ReviewDate                         2403
EnvironmentSatisfaction               5
JobSatisfaction                       5
RelationshipSatisfaction              5
TrainingOpportunitiesWithinYear       3
TrainingOpportunitiesTaken            4
WorkLifeBalance                       5
SelfRating                            3
ManagerRating         

In [10]:
# Display data types for all columns in employee_df
print("Employee DataFrame column data types:")
print(employee_df.dtypes)
print("\n")

# Display data types for all columns in performance_df
print("Performance DataFrame column data types:")
print(performance_df.dtypes)
print("\n")

# Display data types for all columns in education_df
print("Education DataFrame column data types:")
print(education_df.dtypes)
print("\n")

# Display data types for all columns in rating_df
print("Rating DataFrame column data types:")
print(rating_df.dtypes)
print("\n")

# Display data types for all columns in satisfaction_df
print("Satisfaction DataFrame column data types:")
print(satisfaction_df.dtypes)
print("\n")


Employee DataFrame column data types:
EmployeeID                 object
FirstName                  object
LastName                   object
Gender                     object
Age                         int64
BusinessTravel             object
Department                 object
DistanceFromHome (KM)       int64
State                      object
Ethnicity                  object
Education                   int64
EducationField             object
JobRole                    object
MaritalStatus              object
Salary                      int64
StockOptionLevel            int64
OverTime                   object
HireDate                   object
Attrition                  object
YearsAtCompany              int64
YearsInMostRecentRole       int64
YearsSinceLastPromotion     int64
YearsWithCurrManager        int64
HireYear                    int64
AttritionYear               int64
dtype: object


Performance DataFrame column data types:
PerformanceID                              object
Emplo

In [11]:
# Change 'HireDate' in employee_df to datetime format
employee_df['HireDate'] = pd.to_datetime(employee_df['HireDate'])

# Change 'EducationLevelID' in education_df to string (text) format
education_df['EducationLevelID'] = education_df['EducationLevelID'].astype(str)

# Change 'SatisfactionID' in satisfaction_df to string (text) format
satisfaction_df['SatisfactionID'] = satisfaction_df['SatisfactionID'].astype(str)

# Change 'RatingID' in rating_df to string (text) format
rating_df['RatingID'] = rating_df['RatingID'].astype(str)

In [14]:
employee_df.to_csv('E:\\DEPI\\final_project\\HR\\Employee.csv', index=False)
education_df.to_csv('E:\\DEPI\\final_project\\HR\\EducationLevel.csv', index=False)
satisfaction_df.to_csv('E:\\DEPI\\final_project\\HR\\SatisfiedLevel.csv', index=False)
rating_df.to_csv('E:\\DEPI\\final_project\\HR\\RatingLevel.csv', index=False)
performance_df.to_csv('E:\\DEPI\\final_project\\HR\\PerformanceRating.csv', index=False)
